In [1]:
import pandas as pd
import numpy as np
import os

# Google Drive is mounted
from google.colab import drive
drive.mount('/content/drive')

# inspections.csv file is loaded
insp_url = 'https://drive.google.com/uc?id=1Jy03Ay7VnpEdpJEsEarV59K7dLO5efPd'
insp_path = '/content/drive/MyDrive/inspections.csv'
insp = pd.read_csv(insp_url)
insp.to_csv(insp_path, index=False)

# violations.csv file is loaded
viol_url = 'https://drive.google.com/uc?id=1cX7yl58A3tRVkQ_Ta1tNbA9rMm65ZJN_'
viol_path = '/content/drive/MyDrive/violations.csv'
viol = pd.read_csv(viol_url)
viol.to_csv(viol_path, index=False)

# 1.  Determines the following: file format, file encoding, and file size
inspections_info = {
    'Format': insp_url.split('.')[-1],
    'Encoding': 'utf-8',
    'Size': os.path.getsize(insp_path)
}
violations_info = {
    'Format': viol_url.split('.')[-1],
    'Encoding': 'utf-8',
    'Size': os.path.getsize(viol_path)
}

print("File format, encoding, and size of inspections.csv:")
print(inspections_info)
print()
print("File format, encoding, and size of violations.csv:")
print(violations_info)
print()

# 2. Prints out the granularity of each file
print("Granularity of inspections.csv:", len(insp), "rows")
print("Granularity of violations.csv:", len(viol), "rows")

# 3. Focus on inspections that happened only in 2017. Counts the number of violations in each category.
insp['date'] = pd.to_datetime(insp['date'], format='%Y%m%d')
insp_2017 = insp[insp['date'].dt.year == 2017].copy()

viol['date'] = pd.to_datetime(viol['date'], format='%Y%m%d')

merged = pd.merge(insp_2017, viol, on=['business_id', 'date'], how='inner')
violation_counts = merged['description'].value_counts()

print("Top 20 Violations in 2017:")
print(violation_counts.head(20))

# 4. New feature "num_vio" is created. Determines the number of violations of a business at the same time.
num_vio = merged.groupby('business_id').size()
insp_2017['num_vio'] = insp_2017['business_id'].map(num_vio)

print("Updated inspections dataframe:")
print(insp_2017.head())

# 5. Determines how many NaN values exist in the dataframe. Provides answers as to how to address the missing data.
num_nan = insp_2017['num_vio'].isna().sum()
print("Number of NaN values in num_vio column:", num_nan)

# Missing data in the dataset has been addressed by mean imputation
mean_num_vio = np.mean(insp_2017['num_vio'])
insp_2017['num_vio'].fillna(mean_num_vio, inplace=True)

print("DataFrame containing inspections data has been updated with imputed values for missing entries:")
print(insp_2017.head())



Mounted at /content/drive
File format, encoding, and size of inspections.csv:
{'Format': 'com/uc?id=1Jy03Ay7VnpEdpJEsEarV59K7dLO5efPd', 'Encoding': 'utf-8', 'Size': 366543}

File format, encoding, and size of violations.csv:
{'Format': 'com/uc?id=1cX7yl58A3tRVkQ_Ta1tNbA9rMm65ZJN_', 'Encoding': 'utf-8', 'Size': 3530989}

Granularity of inspections.csv: 14222 rows
Granularity of violations.csv: 39042 rows
Top 20 Violations in 2017:
Unclean or degraded floors walls or ceilings                                 725
Unapproved or unmaintained equipment or utensils                             477
Improper food storage                                                        347
Inadequately cleaned or sanitized food contact surfaces                      339
Inadequate and inaccessible handwashing facilities                           329
Moderate risk food holding temperature                                       300
Wiping cloths not clean or properly stored or inadequate sanitizer           299